**Imports**

In [2]:
from citutils.parsers import (
    get_clean_name,
)
import yaml
import json
import citutils.my_types as ty
import os.path
from pathlib import Path
from tinydb import TinyDB, Query

**What I used to make the db initially from the old ./data entities**

In [3]:
# similar to get entities..
def build_db(dir_path: str) -> ty.Entities:
    db = {}
    entity_files = []
    for file in os.listdir(dir_path):
        if file.endswith(".yaml"):
            entity_files.append(os.path.join(dir_path, file))
    for file in entity_files:
        entities = ty.Entities({})
        with open(file, encoding="utf-8") as f:
            data = yaml.safe_load(f)
            if data is None:
                continue
            for entity in data:
                if "table" in entity.keys():
                    entity = add_expanded_outcomes(entity)
                for k, v in entity.items():
                    if k != "table":  # surely this is not the best way to do this...
                        entity[k] = str(v)
                clean_name = get_clean_name(entity["name"])
                assert not clean_name in entities
                entities[clean_name] = entity
            db[Path(file).stem] = entities
    return db


# db = build_db("./data")
# with open("old.json", "w") as f:
#    json.dump(db, f)

**Creates a string doc_id TinyDB (broken)** 

In [4]:
from typing import (
    Iterable,
    List,
    Mapping,
)
from tinydb import TinyDB
from citutils.parsers import get_clean_name


class Document(dict):
    """
    A document stored in the database.

    This class provides a way to access both a document's content and
    its ID using ``doc.doc_id``.
    """

    def __init__(self, value: Mapping, doc_id: str):
        super().__init__(value)
        self.doc_id = doc_id


# class StringDocumentId:
#     def __init__(self, doc_id):
#         self.doc_id = doc_id


class CustomTinyDB(TinyDB):
    def __init__(self, path, document_id_class=str):
        super().__init__(path)
        self.document_id_class = document_id_class

    def insert(self, document: Mapping) -> int:
        """
        Insert a new document into the table.

        :param document: the document to insert
        :returns: the inserted document's ID
        """

        # Make sure the document implements the ``Mapping`` interface
        if not isinstance(document, Mapping):
            raise ValueError("Document is not a Mapping")

        # First, we get the document ID for the new document
        if isinstance(document, Document):
            # For a `Document` object we use the specified ID
            doc_id = document.doc_id
        else:
            if "name" not in Mapping:
                raise KeyError("Mapping has no name")
            doc_id = get_clean_name(document.name)

        # Now, we update the table and add the document
        def updater(table: dict):
            if doc_id in table:
                raise ValueError(f"Document with ID {str(doc_id)} " f"already exists")

            # By calling ``dict(document)`` we convert the data we got to a
            # ``dict`` instance even if it was a different class that
            # implemented the ``Mapping`` interface
            table[doc_id] = dict(document)

        # See below for details on ``Table._update``
        self._update_table(updater)

        return doc_id

    def insert_multiple(self, documents: Iterable[Mapping]) -> List[int]:
        """
        Insert multiple documents into the table.

        :param documents: an Iterable of documents to insert
        :returns: a list containing the inserted documents' IDs
        """
        doc_ids = []

        def updater(table: dict):
            for document in documents:

                # Make sure the document implements the ``Mapping`` interface
                if not isinstance(document, Mapping):
                    raise ValueError("Document is not a Mapping")

                if isinstance(document, Document):
                    # Check if document does not override an existing document
                    if document.doc_id in table:
                        raise ValueError(
                            f"Document with ID {str(document.doc_id)} "
                            f"already exists"
                        )

                    # Store the doc_id, so we can return all document IDs
                    # later. Then save the document with its doc_id and
                    # skip the rest of the current loop
                    doc_id = document.doc_id
                    doc_ids.append(doc_id)
                    table[doc_id] = dict(document)
                    continue
                else:
                    if "name" not in Mapping:
                        raise KeyError("Mapping has no name")
                    doc_id = get_clean_name(document.name)

                # Generate new document ID for this document
                # Store the doc_id, so we can return all document IDs
                # later, then save the document with the new doc_id
                doc_ids.append(doc_id)
                table[doc_id] = dict(document)

        # See below for details on ``Table._update``
        self._update_table(updater)

        return doc_ids


db = CustomTinyDB("db_string.json", document_id_class=str)

**Creates a default TinyDB with integer ids**

In [ ]:
import json
from tinydb import TinyDB, Query
from typing import (
    Iterable,
    List,
    Mapping,
)


# Load your JSON data from a file
with open("db.json", "r") as json_file:
    json_data = json.load(json_file)

# Create a TinyDB database
db = TinyDB("db_default.json")


class Document(dict):
    """
    A document stored in the database.

    This class provides a way to access both a document's content and
    its ID using ``doc.doc_id``.
    """

    def __init__(self, value: Mapping, doc_id: int):
        super().__init__(value)
        self.doc_id = doc_id


i = 0
# Iterate through the JSON data and insert into the database
for category, items in json_data.items():
    for item_id, item_data in items.items():
        item_data["clean_name"] = item_id
        # Create a new Document with an integer doc_id
        db.insert(
            Document(item_data, doc_id=i),
        )
        i += 1
        # Use 0 as a placeholder doc_id
        # Update the doc_id of the inserted Document

**misc. json finagling** 

In [ ]:
# import json

# # i ran this on a lot of fields as they came up...
# # including: meta_tags, scores, requirements... tags, attacks

# def split_meta_tags(obj):
#     if isinstance(obj, list):
#         return [split_meta_tags(item) for item in obj]
#     elif isinstance(obj, dict):
#         for key, value in obj.items():
#             if key == "skills" and isinstance(value, str):
#                 obj[key] = [tag.strip() for tag in value.split(",")]
#             else:
#                 obj[key] = split_meta_tags(value)
#     return obj

# # Load the input JSON file
# with open("db.json", "r") as infile:
#     input_data = json.load(infile)

# # Traverse the JSON and split "meta_tags" as needed
# modified_data = split_meta_tags(input_data)

# # Write the modified JSON to a new file
# with open("output.json", "w") as outfile:
#     json.dump(modified_data, outfile, indent=2)

In [ ]:
# import json 

# def meta_tagger(obj):
#     for k,v in obj:
#         if k != "old":
#             continue
#         else:
            
    
# with open("source copy.json", "r") as infile:
#     input_data = json.load(infile)

ModuleNotFoundError: No module named 'citutils.commands'